In [ ]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.parallel import DataParallel
from torchvision import transforms
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
from transformers import SegformerForSemanticSegmentation, SegformerConfig
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm import tqdm
import json
from pathlib import Path
import time
import warnings
warnings.filterwarnings('ignore')

# GPU 설정
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2"  # GPU 0,1,2 사용
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())
torch.cuda.manual_seed_all(42)

In [ ]:
# ===============================
# 기존 코드 (데이터 매칭, 데이터셋, 변환)
# ===============================

def match_image_mask_pairs(original_root, mask_root):
    """원본 이미지와 마스크 파일을 매칭"""
    print("="*60)
    print("📊 데이터 매칭 및 수집")
    print("="*60)
    print(f"🔍 데이터 매칭 시작")
    print(f"  원본 이미지: {original_root}")
    print(f"  마스크 이미지: {mask_root}")
    
    image_paths = []
    mask_paths = []
    
    original_root = Path(original_root)
    mask_root = Path(mask_root)
    
    # 폴더 매핑 테이블
    folder_mapping = {
        'Fr5_intertek_1st_250526': '1st',
        'Fr5_intertek_2nd_250526': '2nd', 
        'Fr5_intertek_3rd_250526': '3rd',
        'Fr5_intertek_4th_250526': '4th',
        'Fr5_intertek_5th_250526': '5th',
        'Fr5_intertek_6th_250526': '6th',
        'Fr5_intertek_7th_250526': '7th'
    }
    
    total_pairs = 0
    
    for original_folder, mask_folder in folder_mapping.items():
        original_folder_path = original_root / original_folder
        mask_folder_path = mask_root / mask_folder
        
        if not original_folder_path.exists():
            print(f"⚠️  원본 폴더 없음: {original_folder_path}")
            continue
            
        if not mask_folder_path.exists():
            print(f"⚠️  마스크 폴더 없음: {mask_folder_path}")
            continue
            
        print(f"📁 처리 중: {original_folder} → {mask_folder}")
        
        # left, right, top 시점별 처리
        for view in ['left', 'right', 'top']:
            original_view_path = original_folder_path / view
            mask_view_path = mask_folder_path / view / 'masks'
            
            if not original_view_path.exists():
                print(f"  ⚠️  원본 {view} 폴더 없음")
                continue
                
            if not mask_view_path.exists():
                print(f"  ⚠️  마스크 {view}/masks 폴더 없음: {mask_view_path}")
                continue
            
            view_pairs = 0
            unmatched_samples = []
            
            # 원본 이미지들 찾기
            for img_ext in ['*.jpg', '*.jpeg', '*.JPG', '*.JPEG']:
                for original_img_path in original_view_path.glob(img_ext):
                    # 마스크 파일명 패턴들 시도
                    possible_mask_names = [
                        f"{original_img_path.stem}_mask.png",
                        f"{original_img_path.stem}.png",
                        f"{original_img_path.name.replace('.jpg', '_mask.png')}",
                        f"{original_img_path.name.replace('.JPG', '_mask.png')}",
                        f"{original_img_path.name.replace('.jpeg', '_mask.png')}",
                        f"{original_img_path.name.replace('.JPEG', '_mask.png')}"
                    ]
                    
                    found = False
                    for possible_name in possible_mask_names:
                        possible_mask_path = mask_view_path / possible_name
                        if possible_mask_path.exists():
                            image_paths.append(str(original_img_path))
                            mask_paths.append(str(possible_mask_path))
                            view_pairs += 1
                            found = True
                            break
                    
                    if not found:
                        unmatched_samples.append(original_img_path.name)
            
            if view_pairs > 0:
                print(f"  ✅ {view}: {view_pairs}개 매칭 쌍")
                total_pairs += view_pairs
                
                if unmatched_samples:
                    print(f"    ⚠️  매칭 실패 ({len(unmatched_samples)}개): {unmatched_samples[:3]}")
            else:
                print(f"  ❌ {view}: 매칭 실패")
    
    print(f"\n🎉 총 {len(image_paths)}개의 이미지-마스크 쌍을 찾았습니다!")
    return image_paths, mask_paths

In [ ]:
class RobotArmDataset(Dataset):
    def __init__(self, image_paths, mask_paths, transform=None, is_train=True):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.transform = transform
        self.is_train = is_train
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        # 원본 RGB 이미지 로드
        image_path = self.image_paths[idx]
        image = cv2.imread(image_path)
        
        if image is None:
            raise ValueError(f"이미지 로드 실패: {image_path}")
            
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # 마스크 이미지 로드
        mask_path = self.mask_paths[idx]
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        
        if mask is None:
            raise ValueError(f"마스크 로드 실패: {mask_path}")
        
        # 마스크를 이진화 (0: 배경, 1: 로봇팔)
        mask = (mask > 127).astype(np.uint8)
        
        # 변환 적용
        if self.transform:
            transformed = self.transform(image=image, mask=mask)
            image = transformed['image']
            mask = transformed['mask']
        
        return {
            'image': image,
            'mask': mask.long(),
            'image_path': image_path,
            'mask_path': mask_path
        }

In [ ]:
def get_train_transforms(image_size=512, augmentation_level='moderate'):
    """
    데이터 증강 설정
    augmentation_level: 'light', 'moderate', 'heavy'
    """
    
    if augmentation_level == 'light':
        # 가벼운 증강 (빠른 훈련, 안정적)
        return A.Compose([
            A.Resize(image_size, image_size),
            A.HorizontalFlip(p=0.5),
            A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=0.3),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ])
    
    elif augmentation_level == 'moderate':
        # 중간 증강 (균형잡힌 성능, 권장)
        return A.Compose([
            A.Resize(image_size, image_size),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.2),
            A.RandomRotate90(p=0.5),
            A.ShiftScaleRotate(
                shift_limit=0.1, 
                scale_limit=0.1, 
                rotate_limit=15, 
                p=0.5
            ),
            A.RandomBrightnessContrast(
                brightness_limit=0.2, 
                contrast_limit=0.2, 
                p=0.5
            ),
            A.HueSaturationValue(
                hue_shift_limit=10, 
                sat_shift_limit=20, 
                val_shift_limit=10, 
                p=0.3
            ),
            A.GaussNoise(var_limit=(10, 50), p=0.3),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ])
    
    elif augmentation_level == 'heavy':
        # 강한 증강 (로봇팔 특화, 더 robust한 모델)
        return A.Compose([
            A.Resize(image_size, image_size),
            
            # 기하학적 변환 (로봇팔의 다양한 자세)
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.3),
            A.RandomRotate90(p=0.5),
            A.Rotate(limit=30, p=0.5),  # 더 큰 회전각
            A.ShiftScaleRotate(
                shift_limit=0.15, 
                scale_limit=0.15, 
                rotate_limit=20, 
                p=0.6
            ),
            
            # 색상/조명 변환 (다양한 환경 조건)
            A.RandomBrightnessContrast(
                brightness_limit=0.3, 
                contrast_limit=0.3, 
                p=0.6
            ),
            A.HueSaturationValue(
                hue_shift_limit=15, 
                sat_shift_limit=30, 
                val_shift_limit=15, 
                p=0.4
            ),
            A.RandomGamma(gamma_limit=(80, 120), p=0.3),
            A.CLAHE(clip_limit=2.0, p=0.3),  # 대비 개선
            
            # 노이즈 및 블러 (카메라 품질 시뮬레이션)
            A.GaussNoise(var_limit=(10, 80), p=0.4),
            A.GaussianBlur(blur_limit=3, p=0.2),
            A.MotionBlur(blur_limit=3, p=0.2),
            
            # 가려짐 시뮬레이션 (실제 환경의 장애물)
            A.CoarseDropout(
                max_holes=3, 
                max_height=32, 
                max_width=32, 
                min_holes=1, 
                min_height=8, 
                min_width=8, 
                p=0.3
            ),
            
            # 그림자 효과 (조명 변화) - 일부 버전에서 지원하지 않을 수 있음
            # A.RandomShadow(
            #     shadow_roi=(0, 0.5, 1, 1),
            #     num_shadows_lower=1,
            #     num_shadows_upper=2,
            #     shadow_dimension=5,
            #     p=0.3
            # ),
            
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ])
    
    else:
        raise ValueError("augmentation_level must be 'light', 'moderate', or 'heavy'")

def get_val_transforms(image_size=512):
    return A.Compose([
        A.Resize(image_size, image_size),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ])

def debug_tensor_shapes(images, masks, outputs, step_name=""):
    """텐서 크기 디버깅용 함수"""
    print(f"\n🔍 {step_name} 텐서 크기:")
    print(f"  입력 이미지: {images.shape}")
    print(f"  타겟 마스크: {masks.shape}")
    print(f"  모델 출력: {outputs.shape}")
    
    if images.shape[-2:] != outputs.shape[-2:]:
        print(f"  ⚠️ 크기 불일치 감지!")
        print(f"    입력 크기: {images.shape[-2:]}")
        print(f"    출력 크기: {outputs.shape[-2:]}")
    else:
        print(f"  ✅ 크기 일치 확인")
    """데이터 품질 검증"""
    print(f"\n🔍 데이터 품질 검증 (샘플 {num_samples}개)")
    print("="*50)
    
    for i in range(min(num_samples, len(image_paths))):
        img_path = image_paths[i]
        mask_path = mask_paths[i]
        
        print(f"\n📋 샘플 {i+1}:")
        print(f"  원본: {Path(img_path).name}")
        print(f"  마스크: {Path(mask_path).name}")
        
        try:
            img = cv2.imread(img_path)
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            
            if img is None:
                print(f"  ❌ 원본 이미지 로드 실패")
                continue
                
            if mask is None:
                print(f"  ❌ 마스크 이미지 로드 실패")
                continue
            
            print(f"  📏 원본 크기: {img.shape}")
            print(f"  📏 마스크 크기: {mask.shape}")
            
            unique_values = np.unique(mask)
            print(f"  🎨 마스크 픽셀값: {unique_values}")
            
            robot_pixels = np.sum(mask > 127)
            total_pixels = mask.shape[0] * mask.shape[1]
            robot_ratio = robot_pixels / total_pixels * 100
            print(f"  🤖 로봇팔 비율: {robot_ratio:.1f}%")
            
            print(f"  ✅ 정상")
            
        except Exception as e:
            print(f"  ❌ 오류: {e}")

In [ ]:
# ===============================
# 성능 지표 계산 함수들
# ===============================

def calculate_iou(pred, target, num_classes=2):
    """IoU 계산"""
    ious = []
    for cls in range(num_classes):
        pred_cls = (pred == cls)
        target_cls = (target == cls)
        
        intersection = torch.logical_and(pred_cls, target_cls).sum()
        union = torch.logical_or(pred_cls, target_cls).sum()
        
        if union == 0:
            iou = 1.0  # 해당 클래스가 없는 경우
        else:
            iou = intersection.float() / union.float()
        
        ious.append(iou)
    
    return torch.stack(ious)

def calculate_dice(pred, target, num_classes=2):
    """Dice Score 계산"""
    dices = []
    for cls in range(num_classes):
        pred_cls = (pred == cls)
        target_cls = (target == cls)
        
        intersection = torch.logical_and(pred_cls, target_cls).sum()
        total = pred_cls.sum() + target_cls.sum()
        
        if total == 0:
            dice = 1.0  # 해당 클래스가 없는 경우
        else:
            dice = (2.0 * intersection.float()) / total.float()
        
        dices.append(dice)
    
    return torch.stack(dices)

def calculate_pixel_accuracy(pred, target):
    """Pixel Accuracy 계산"""
    correct = (pred == target).sum()
    total = target.numel()
    return correct.float() / float(total)

In [ ]:
# ===============================
# SegFormer 모델 정의
# ===============================

class SegFormerForRobotArm(nn.Module):
    def __init__(self, num_classes=2, model_name="nvidia/mit-b2"):
        super().__init__()
        self.num_classes = num_classes
        
        # SegFormer 모델 로드 (pretrained MiT 백본 포함)
        # model_name 옵션:
        # - nvidia/mit-b0: 3.7M params (빠름, 가벼움)
        # - nvidia/mit-b1: 14M params  
        # - nvidia/mit-b2: 25M params (권장, 균형점)
        # - nvidia/mit-b3: 45M params (더 높은 성능)
        # - nvidia/mit-b4: 62M params
        # - nvidia/mit-b5: 82M params (최고 성능, 느림)
        
        print(f"🏗️ SegFormer 모델 로딩: {model_name}")
        self.segformer = SegformerForSemanticSegmentation.from_pretrained(
            model_name,
            num_labels=num_classes,
            ignore_mismatched_sizes=True
        )
        
        # 모델 파라미터 수 출력
        total_params = sum(p.numel() for p in self.segformer.parameters())
        trainable_params = sum(p.numel() for p in self.segformer.parameters() if p.requires_grad)
        print(f"📊 총 파라미터: {total_params:,}")
        print(f"📊 훈련 가능 파라미터: {trainable_params:,}")
        
    def forward(self, pixel_values):
        outputs = self.segformer(pixel_values=pixel_values)
        logits = outputs.logits
        
        # SegFormer 출력을 입력 크기로 업샘플링
        # 입력: [B, C, H, W] -> 출력: [B, num_classes, H, W]
        upsampled_logits = nn.functional.interpolate(
            logits,
            size=pixel_values.shape[-2:],  # (H, W)
            mode='bilinear',
            align_corners=False
        )
        
        return upsampled_logits

In [ ]:
# ===============================
# 손실 함수 정의
# ===============================

class CombinedLoss(nn.Module):
    def __init__(self, alpha=0.7, beta=0.3):
        super().__init__()
        self.alpha = alpha  # CrossEntropyLoss 가중치
        self.beta = beta    # DiceLoss 가중치
        self.ce_loss = nn.CrossEntropyLoss()
        
    def dice_loss(self, pred, target, smooth=1e-6):
        """Dice Loss 계산"""
        pred = torch.softmax(pred, dim=1)
        
        dice_losses = []
        for cls in range(pred.shape[1]):
            pred_cls = pred[:, cls, :, :]
            target_cls = (target == cls).float()
            
            intersection = (pred_cls * target_cls).sum(dim=(1, 2))
            union = pred_cls.sum(dim=(1, 2)) + target_cls.sum(dim=(1, 2))
            
            dice = (2.0 * intersection + smooth) / (union + smooth)
            dice_loss = 1.0 - dice
            dice_losses.append(dice_loss.mean())
        
        return sum(dice_losses) / len(dice_losses)
    
    def forward(self, pred, target):
        ce_loss = self.ce_loss(pred, target)
        dice_loss = self.dice_loss(pred, target)
        
        total_loss = self.alpha * ce_loss + self.beta * dice_loss
        return total_loss, ce_loss, dice_loss


In [ ]:
# ===============================
# 훈련 및 검증 함수
# ===============================

def train_epoch(model, train_loader, criterion, optimizer, device, epoch):
    """한 에포크 훈련"""
    model.train()
    running_loss = 0.0
    running_ce_loss = 0.0
    running_dice_loss = 0.0
    running_iou = 0.0
    running_dice_score = 0.0
    running_pixel_acc = 0.0
    
    progress_bar = tqdm(train_loader, desc=f'Train Epoch {epoch}')
    
    for batch_idx, batch in enumerate(progress_bar):
        images = batch['image'].to(device)
        masks = batch['mask'].to(device)
        
        optimizer.zero_grad()
        
        # 모델 예측
        outputs = model(images)
        
        # 첫 번째 배치에서 크기 확인 (디버깅용)
        if batch_idx == 0 and epoch == 1:
            print(f"\n🔍 첫 번째 훈련 배치 텐서 크기:")
            print(f"  입력 이미지: {images.shape}")
            print(f"  타겟 마스크: {masks.shape}")
            print(f"  모델 출력: {outputs.shape}")
            
            if images.shape[-2:] != outputs.shape[-2:]:
                print(f"  ⚠️ 크기 불일치 감지!")
                print(f"    입력 크기: {images.shape[-2:]}")
                print(f"    출력 크기: {outputs.shape[-2:]}")
            else:
                print(f"  ✅ 크기 일치 확인")
        
        # 손실 계산
        loss, ce_loss, dice_loss = criterion(outputs, masks)
        
        # 역전파
        loss.backward()
        optimizer.step()
        
        # 성능 지표 계산
        with torch.no_grad():
            pred_masks = torch.argmax(outputs, dim=1)
            
            iou = calculate_iou(pred_masks, masks).mean()
            dice_score = calculate_dice(pred_masks, masks).mean()
            pixel_acc = calculate_pixel_accuracy(pred_masks, masks)
            
            running_loss += loss.item()
            running_ce_loss += ce_loss.item()
            running_dice_loss += dice_loss.item()
            running_iou += iou.item()
            running_dice_score += dice_score.item()
            running_pixel_acc += pixel_acc.item()
        
        # 진행률 업데이트
        if batch_idx % 10 == 0:
            progress_bar.set_postfix({
                'Loss': f'{loss.item():.4f}',
                'IoU': f'{iou.item():.4f}',
                'Dice': f'{dice_score.item():.4f}',
                'PixelAcc': f'{pixel_acc.item():.4f}'
            })
    
    num_batches = len(train_loader)
    return {
        'loss': running_loss / num_batches,
        'ce_loss': running_ce_loss / num_batches,
        'dice_loss': running_dice_loss / num_batches,
        'iou': running_iou / num_batches,
        'dice_score': running_dice_score / num_batches,
        'pixel_accuracy': running_pixel_acc / num_batches
    }

def validate_epoch(model, val_loader, criterion, device, epoch):
    """한 에포크 검증"""
    model.eval()
    
    running_loss = 0.0
    running_ce_loss = 0.0
    running_dice_loss = 0.0
    running_iou = 0.0
    running_dice_score = 0.0
    running_pixel_acc = 0.0
    
    progress_bar = tqdm(val_loader, desc=f'Val Epoch {epoch}')
    
    with torch.no_grad():
        for batch in progress_bar:
            images = batch['image'].to(device)
            masks = batch['mask'].to(device)
            
            # 모델 예측
            outputs = model(images)
            
            # 손실 계산
            loss, ce_loss, dice_loss = criterion(outputs, masks)
            
            # 성능 지표 계산
            pred_masks = torch.argmax(outputs, dim=1)
            
            iou = calculate_iou(pred_masks, masks).mean()
            dice_score = calculate_dice(pred_masks, masks).mean()
            pixel_acc = calculate_pixel_accuracy(pred_masks, masks)
            
            running_loss += loss.item()
            running_ce_loss += ce_loss.item()
            running_dice_loss += dice_loss.item()
            running_iou += iou.item()
            running_dice_score += dice_score.item()
            running_pixel_acc += pixel_acc.item()
            
            progress_bar.set_postfix({
                'Loss': f'{loss.item():.4f}',
                'IoU': f'{iou.item():.4f}',
                'Dice': f'{dice_score.item():.4f}',
                'PixelAcc': f'{pixel_acc.item():.4f}'
            })
    
    num_batches = len(val_loader)
    return {
        'loss': running_loss / num_batches,
        'ce_loss': running_ce_loss / num_batches,
        'dice_loss': running_dice_loss / num_batches,
        'iou': running_iou / num_batches,
        'dice_score': running_dice_score / num_batches,
        'pixel_accuracy': running_pixel_acc / num_batches
    }

In [ ]:
# ===============================
# 결과 시각화 함수
# ===============================

def visualize_predictions(model, val_loader, device, num_samples=3):
    """예측 결과 시각화"""
    model.eval()
    
    # 역정규화를 위한 함수
    def denormalize(tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]):
        for t, m, s in zip(tensor, mean, std):
            t.mul_(s).add_(m)
        return torch.clamp(tensor, 0, 1)
    
    fig, axes = plt.subplots(num_samples, 3, figsize=(15, 5*num_samples))
    if num_samples == 1:
        axes = axes.reshape(1, -1)
    
    with torch.no_grad():
        for i, batch in enumerate(val_loader):
            if i >= num_samples:
                break
                
            images = batch['image'].to(device)
            masks = batch['mask'].to(device)
            
            outputs = model(images)
            pred_masks = torch.argmax(outputs, dim=1)
            
            # 첫 번째 이미지만 시각화
            image = images[0].cpu()
            mask = masks[0].cpu().numpy()
            pred_mask = pred_masks[0].cpu().numpy()
            
            # 이미지 역정규화
            image = denormalize(image)
            image = image.permute(1, 2, 0).numpy()
            
            # 원본 이미지
            axes[i, 0].imshow(image)
            axes[i, 0].set_title(f'Original Image {i+1}')
            axes[i, 0].axis('off')
            
            # 실제 마스크
            axes[i, 1].imshow(mask, cmap='gray')
            axes[i, 1].set_title(f'Ground Truth Mask {i+1}')
            axes[i, 1].axis('off')
            
            # 예측 마스크
            axes[i, 2].imshow(pred_mask, cmap='gray')
            axes[i, 2].set_title(f'Predicted Mask {i+1}')
            axes[i, 2].axis('off')
    
    plt.tight_layout()
    plt.savefig('prediction_results.png', dpi=150, bbox_inches='tight')
    plt.show()


In [ ]:
# ===============================
# 메인 훈련 함수
# ===============================

def main():
    # 하이퍼파라미터 설정
    BATCH_SIZE = 8  # GPU 메모리에 따라 조정
    IMAGE_SIZE = 512
    LEARNING_RATE = 1e-4
    NUM_EPOCHS = 50
    PATIENCE = 10  # Early stopping
    
    # 모델 크기 선택 (성능 vs 속도 트레이드오프)
    MODEL_VARIANTS = {
        'small': 'nvidia/mit-b0',    # 빠름, 가벼움
        'medium': 'nvidia/mit-b2',   # 권장 (균형점)
        'large': 'nvidia/mit-b3',    # 높은 성능
        'xlarge': 'nvidia/mit-b4'    # 최고 성능 (메모리 많이 필요)
    }
    
    # 증강 레벨 선택
    AUGMENTATION_LEVELS = {
        'light': '가벼운 증강 (빠른 훈련)',
        'moderate': '중간 증강 (권장)',
        'heavy': '강한 증강 (더 robust한 모델)'
    }
    
    # 설정 (필요에 따라 변경)
    MODEL_SIZE = 'medium'  # 'small', 'medium', 'large', 'xlarge'
    AUGMENTATION_LEVEL = 'moderate'  # 'light', 'moderate', 'heavy'
    
    print("🚀 SegFormer 로봇팔 Segmentation 훈련 시작")
    print("="*60)
    print(f"🏗️ 모델 크기: {MODEL_SIZE} ({MODEL_VARIANTS[MODEL_SIZE]})")
    print(f"🔄 데이터 증강: {AUGMENTATION_LEVEL} ({AUGMENTATION_LEVELS[AUGMENTATION_LEVEL]})")
    
    # 1. 데이터 로딩 및 매칭
    original_images_root = '/home/ibom002/dataset'
    mask_images_root = '/home/ibom002/dataset'
    
    image_paths, mask_paths = match_image_mask_pairs(
        original_images_root, 
        mask_images_root
    )
    
    if len(image_paths) == 0:
        print("❌ 매칭된 데이터가 없습니다.")
        return
    
    # 2. 데이터 품질 검증
    # verify_data_quality(image_paths, mask_paths, num_samples=3)
    
    # 3. Train/Validation 분할 (stratified by view)
    # 시점 정보 추출 (경로에서)
    views = []
    for img_path in image_paths:
        if 'left' in img_path:
            views.append('left')
        elif 'right' in img_path:
            views.append('right')
        elif 'top' in img_path:
            views.append('top')
        else:
            views.append('unknown')
    
    train_img_paths, val_img_paths, train_mask_paths, val_mask_paths = train_test_split(
        image_paths, mask_paths,
        test_size=0.2,
        random_state=42,
        stratify=views
    )
    
    print(f"\n📊 데이터 분할 결과:")
    print(f"  훈련 데이터: {len(train_img_paths)}개")
    print(f"  검증 데이터: {len(val_img_paths)}개")
    
    # 4. 데이터셋 및 데이터로더 생성
    train_transforms = get_train_transforms(IMAGE_SIZE, AUGMENTATION_LEVEL)
    val_transforms = get_val_transforms(IMAGE_SIZE)
    
    train_dataset = RobotArmDataset(
        train_img_paths, train_mask_paths, 
        transform=train_transforms, 
        is_train=True
    )
    
    val_dataset = RobotArmDataset(
        val_img_paths, val_mask_paths, 
        transform=val_transforms, 
        is_train=False
    )
    
    train_loader = DataLoader(
        train_dataset, 
        batch_size=BATCH_SIZE, 
        shuffle=True, 
        num_workers=4,
        pin_memory=True
    )
    
    val_loader = DataLoader(
        val_dataset, 
        batch_size=BATCH_SIZE, 
        shuffle=False, 
        num_workers=4,
        pin_memory=True
    )
    
    print(f"\n📈 배치 정보:")
    print(f"  훈련 배치 수: {len(train_loader)}")
    print(f"  검증 배치 수: {len(val_loader)}")
    print(f"  배치 크기: {BATCH_SIZE}")
    
    # 5. 모델 생성
    print(f"\n🏗️ 모델 생성 중...")
    model = SegFormerForRobotArm(
        num_classes=2, 
        model_name=MODEL_VARIANTS[MODEL_SIZE]
    )
    
    # Multi-GPU 사용
    if torch.cuda.device_count() > 1:
        print(f"🖥️ {torch.cuda.device_count()}개 GPU 사용")
        model = DataParallel(model)
    
    model = model.to(device)
    
    # 6. 손실 함수 및 옵티마이저
    criterion = CombinedLoss(alpha=0.7, beta=0.3)
    optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
    
    # 학습률 스케줄러
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=5
    )
    
    # 7. 훈련 기록을 위한 변수들
    train_history = []
    val_history = []
    best_val_iou = 0.0
    best_epoch = 0
    patience_counter = 0
    
    print(f"\n🎯 훈련 시작!")
    print(f"  총 에포크: {NUM_EPOCHS}")
    print(f"  학습률: {LEARNING_RATE}")
    print(f"  Early Stopping 기준: {PATIENCE} 에포크")
    
    # 8. 훈련 루프
    for epoch in range(1, NUM_EPOCHS + 1):
        start_time = time.time()
        
        # 훈련
        train_metrics = train_epoch(model, train_loader, criterion, optimizer, device, epoch)
        
        # 검증
        val_metrics = validate_epoch(model, val_loader, criterion, device, epoch)
        
        # 학습률 스케줄러 업데이트
        old_lr = optimizer.param_groups[0]['lr']
        scheduler.step(val_metrics['loss'])
        new_lr = optimizer.param_groups[0]['lr']
        
        # 학습률 변경 감지 및 출력
        if new_lr < old_lr:
            print(f"  📉 학습률 감소: {old_lr:.2e} → {new_lr:.2e}")
        
        # 기록 저장
        train_history.append(train_metrics)
        val_history.append(val_metrics)
        
        epoch_time = time.time() - start_time
        
        # 결과 출력
        print(f"\n📊 Epoch {epoch}/{NUM_EPOCHS} 결과 (소요시간: {epoch_time:.2f}초)")
        print(f"  Train - Loss: {train_metrics['loss']:.4f}, IoU: {train_metrics['iou']:.4f}, "
              f"Dice: {train_metrics['dice_score']:.4f}, PixelAcc: {train_metrics['pixel_accuracy']:.4f}")
        print(f"  Val   - Loss: {val_metrics['loss']:.4f}, IoU: {val_metrics['iou']:.4f}, "
              f"Dice: {val_metrics['dice_score']:.4f}, PixelAcc: {val_metrics['pixel_accuracy']:.4f}")
        
        # 최고 성능 모델 저장
        if val_metrics['iou'] > best_val_iou:
            best_val_iou = val_metrics['iou']
            best_epoch = epoch
            patience_counter = 0
            
            # 모델 저장
            if isinstance(model, DataParallel):
                model_state = model.module.state_dict()
            else:
                model_state = model.state_dict()
            
            torch.save({
                'epoch': epoch,
                'model_state_dict': model_state,
                'optimizer_state_dict': optimizer.state_dict(),
                'val_iou': val_metrics['iou'],
                'val_dice': val_metrics['dice_score'],
                'train_history': train_history,
                'val_history': val_history
            }, 'best_segformer_robot_arm.pth')
            
            print(f"  🎉 새로운 최고 성능! IoU: {best_val_iou:.4f} (Epoch {epoch})")
            
        else:
            patience_counter += 1
            print(f"  ⏳ 성능 개선 없음 ({patience_counter}/{PATIENCE})")
        
        # Early Stopping
        if patience_counter >= PATIENCE:
            print(f"\n🛑 Early Stopping! 최고 성능: IoU {best_val_iou:.4f} (Epoch {best_epoch})")
            break
        
        # 매 5 에포크마다 예측 결과 시각화
        if epoch % 5 == 0:
            print(f"\n🖼️ 예측 결과 시각화 (Epoch {epoch})")
            visualize_predictions(model, val_loader, device, num_samples=3)
    
    # 9. 훈련 완료 후 최종 결과
    print(f"\n🎊 훈련 완료!")
    print(f"  최고 성능: IoU {best_val_iou:.4f} (Epoch {best_epoch})")
    print(f"  모델 저장 위치: best_segformer_robot_arm.pth")
    

    
    # 11. 최종 예측 결과 시각화
    print(f"\n🖼️ 최종 예측 결과")
    visualize_predictions(model, val_loader, device, num_samples=5)
    
    # 12. 성능 지표 요약
    # print_final_metrics(val_history, best_epoch)


In [ ]:
if __name__ == "__main__":
    # 메인 훈련 실행
    main()

In [ ]:
# ===============================
# 간단한 SegFormer 모델 성능 평가 코드
# ===============================

import torch
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

def load_best_model(checkpoint_path, device):
    """저장된 최고 성능 모델 로드"""
    print(f"📂 모델 로딩: {checkpoint_path}")
    
    # 모델 생성 (기존 클래스 사용)
    model = SegFormerForRobotArm(num_classes=2, model_name="nvidia/mit-b2")
    
    # 체크포인트 로드
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    model = model.to(device)
    model.eval()
    
    print(f"✅ 모델 로드 완료 (Epoch {checkpoint['epoch']}, IoU: {checkpoint['val_iou']:.4f})")
    return model, checkpoint

def calculate_metrics(pred_masks, true_masks):
    """기본 성능 지표 계산"""
    # 전체 픽셀 정확도
    pixel_acc = (pred_masks == true_masks).float().mean()
    
    # 클래스별 IoU 계산
    ious = []
    dices = []
    
    for cls in range(2):  # 배경(0), 로봇팔(1)
        pred_cls = (pred_masks == cls)
        true_cls = (true_masks == cls)
        
        # IoU
        intersection = (pred_cls & true_cls).sum().float()
        union = (pred_cls | true_cls).sum().float()
        iou = intersection / union if union > 0 else 1.0
        
        # Dice Score
        dice = (2.0 * intersection) / (pred_cls.sum() + true_cls.sum()) if (pred_cls.sum() + true_cls.sum()) > 0 else 1.0
        
        ious.append(iou.item())
        dices.append(dice.item())
    
    return {
        'pixel_accuracy': pixel_acc.item(),
        'background_iou': ious[0],
        'robot_arm_iou': ious[1],
        'mean_iou': np.mean(ious),
        'background_dice': dices[0],
        'robot_arm_dice': dices[1],
        'mean_dice': np.mean(dices)
    }

def evaluate_model(model, val_loader, device):
    """모델 성능 평가"""
    print("\n🔍 모델 성능 평가 시작...")
    
    model.eval()
    all_metrics = []
    
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="평가 중"):
            images = batch['image'].to(device)
            masks = batch['mask'].to(device)
            
            # 예측
            outputs = model(images)
            pred_masks = torch.argmax(outputs, dim=1)
            
            # 메트릭 계산
            metrics = calculate_metrics(pred_masks, masks)
            all_metrics.append(metrics)
    
    # 전체 평균 계산
    avg_metrics = {}
    for key in all_metrics[0].keys():
        avg_metrics[key] = np.mean([m[key] for m in all_metrics])
        avg_metrics[key + '_std'] = np.std([m[key] for m in all_metrics])
    
    return avg_metrics

def visualize_sample_predictions(model, val_loader, device, num_samples=3):
    """샘플 예측 결과 시각화"""
    model.eval()
    
    fig, axes = plt.subplots(num_samples, 3, figsize=(12, 4*num_samples))
    if num_samples == 1:
        axes = axes.reshape(1, -1)
    
    def denormalize(tensor):
        # ImageNet 정규화 역변환
        mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
        std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
        return torch.clamp(tensor * std + mean, 0, 1)
    
    with torch.no_grad():
        for i, batch in enumerate(val_loader):
            if i >= num_samples:
                break
                
            images = batch['image'].to(device)
            masks = batch['mask'].to(device)
            
            outputs = model(images)
            pred_masks = torch.argmax(outputs, dim=1)
            
            # 첫 번째 이미지만 시각화
            image = denormalize(images[0].cpu()).permute(1, 2, 0).numpy()
            true_mask = masks[0].cpu().numpy()
            pred_mask = pred_masks[0].cpu().numpy()
            
            # IoU 계산
            pred_robot = (pred_mask == 1)
            true_robot = (true_mask == 1)
            intersection = np.logical_and(pred_robot, true_robot).sum()
            union = np.logical_or(pred_robot, true_robot).sum()
            iou = intersection / union if union > 0 else 1.0
            
            # 시각화
            axes[i, 0].imshow(image)
            axes[i, 0].set_title(f'Original Image {i+1}')
            axes[i, 0].axis('off')
            
            axes[i, 1].imshow(true_mask, cmap='gray')
            axes[i, 1].set_title('Ground Truth')
            axes[i, 1].axis('off')
            
            axes[i, 2].imshow(pred_mask, cmap='gray')
            axes[i, 2].set_title(f'Prediction (IoU: {iou:.3f})')
            axes[i, 2].axis('off')
    
    plt.tight_layout()
    plt.savefig('evaluation_samples.png', dpi=150, bbox_inches='tight')
    plt.show()

def print_evaluation_results(metrics):
    """평가 결과 출력"""
    print("\n📊 모델 성능 평가 결과")
    print("="*60)
    
    print(f"🎯 전체 성능:")
    print(f"  Pixel Accuracy: {metrics['pixel_accuracy']:.4f} ± {metrics['pixel_accuracy_std']:.4f}")
    print(f"  Mean IoU: {metrics['mean_iou']:.4f} ± {metrics['mean_iou_std']:.4f}")
    print(f"  Mean Dice: {metrics['mean_dice']:.4f} ± {metrics['mean_dice_std']:.4f}")
    
    print(f"\n📋 클래스별 성능:")
    print(f"  배경 (Background):")
    print(f"    IoU: {metrics['background_iou']:.4f} ± {metrics['background_iou_std']:.4f}")
    print(f"    Dice: {metrics['background_dice']:.4f} ± {metrics['background_dice_std']:.4f}")
    
    print(f"  로봇팔 (Robot Arm):")
    print(f"    IoU: {metrics['robot_arm_iou']:.4f} ± {metrics['robot_arm_iou_std']:.4f}")
    print(f"    Dice: {metrics['robot_arm_dice']:.4f} ± {metrics['robot_arm_dice_std']:.4f}")
    
    # 성능 등급
    robot_iou = metrics['robot_arm_iou']
    if robot_iou >= 0.95:
        grade = "🥇 Excellent"
    elif robot_iou >= 0.90:
        grade = "🥈 Very Good"
    elif robot_iou >= 0.85:
        grade = "🥉 Good"
    elif robot_iou >= 0.80:
        grade = "✅ Acceptable"
    else:
        grade = "⚠️ Needs Improvement"
    
    print(f"\n🎖️ 로봇팔 성능 등급: {grade}")

def run_simple_evaluation():
    """간단한 평가 실행"""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    checkpoint_path = 'best_segformer_robot_arm.pth'
    
    # 데이터 로더 생성 (기존 훈련 코드와 동일)
    print("📊 데이터 로딩 중...")
    
    # 기존 훈련 코드에서 사용한 함수들 (import 필요)
    original_images_root = '/home/ibom002/dataset'
    mask_images_root = '/home/ibom002/dataset'
    
    # 데이터 매칭 (기존 함수 사용)
    image_paths, mask_paths = match_image_mask_pairs(
        original_images_root, 
        mask_images_root
    )
    
    # 시점 정보 추출
    views = []
    for img_path in image_paths:
        if 'left' in img_path:
            views.append('left')
        elif 'right' in img_path:
            views.append('right')
        elif 'top' in img_path:
            views.append('top')
        else:
            views.append('unknown')
    
    # Train/Validation 분할 (기존과 동일한 방식)
    from sklearn.model_selection import train_test_split
    train_img_paths, val_img_paths, train_mask_paths, val_mask_paths = train_test_split(
        image_paths, mask_paths,
        test_size=0.2,
        random_state=42,  # 동일한 분할 보장
        stratify=views
    )
    
    # Validation 데이터셋 생성
    val_transforms = get_val_transforms(512)  # 기존 함수 사용
    val_dataset = RobotArmDataset(
        val_img_paths, val_mask_paths, 
        transform=val_transforms, 
        is_train=False
    )
    
    # Validation 데이터로더 생성
    from torch.utils.data import DataLoader
    val_loader = DataLoader(
        val_dataset, 
        batch_size=8, 
        shuffle=False, 
        num_workers=4,
        pin_memory=True
    )
    
    print(f"검증 데이터: {len(val_img_paths)}개")
    
    # 모델 로드
    model, checkpoint = load_best_model(checkpoint_path, device)
    
    # 성능 평가
    metrics = evaluate_model(model, val_loader, device)
    
    # 결과 출력
    print_evaluation_results(metrics)
    
    # 샘플 시각화
    print(f"\n🖼️ 샘플 예측 결과 시각화")
    visualize_sample_predictions(model, val_loader, device, num_samples=3)
    
    return metrics

# 방법 1: 독립적으로 실행 (데이터 로더 새로 생성)
if __name__ == "__main__":
    # 필요한 함수들이 이미 정의되어 있어야 함
    # match_image_mask_pairs, get_val_transforms, RobotArmDataset 등
    
    metrics = run_simple_evaluation()
    print(f"\n🎊 평가 완료!")
    print(f"결과 이미지: evaluation_samples.png")

In [ ]:
import torch
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

def load_best_model(checkpoint_path, device):
    """저장된 최고 성능 모델 로드"""
    print(f"📂 모델 로딩: {checkpoint_path}")
    model = SegFormerForRobotArm(num_classes=2, model_name="nvidia/mit-b2")
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    model = model.to(device)
    model.eval()
    print(f"✅ 모델 로드 완료 (Epoch {checkpoint['epoch']}, IoU: {checkpoint['val_iou']:.4f})")
    return model, checkpoint

def calculate_metrics(pred_masks, true_masks):
    """기본 성능 지표 계산"""
    pixel_acc = (pred_masks == true_masks).float().mean()
    ious, dices = [], []
    for cls in range(2):
        pred_cls = (pred_masks == cls)
        true_cls = (true_masks == cls)
        intersection = (pred_cls & true_cls).sum().float()
        union = (pred_cls | true_cls).sum().float()
        iou = intersection / union if union > 0 else 1.0
        dice = (2.0 * intersection) / (pred_cls.sum() + true_cls.sum()) if (pred_cls.sum() + true_cls.sum()) > 0 else 1.0
        ious.append(iou.item())
        dices.append(dice.item())
    return {
        'pixel_accuracy': pixel_acc.item(),
        'background_iou': ious[0],
        'robot_arm_iou': ious[1],
        'mean_iou': np.mean(ious),
        'background_dice': dices[0],
        'robot_arm_dice': dices[1],
        'mean_dice': np.mean(dices)
    }

def evaluate_model(model, val_loader, device):
    """모델 성능 평가"""
    print("\n🔍 모델 성능 평가 시작...")
    model.eval()
    all_metrics = []
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="평가 중"):
            images = batch['image'].to(device)
            masks = batch['mask'].to(device)
            outputs = model(images)
            pred_masks = torch.argmax(outputs, dim=1)
            metrics = calculate_metrics(pred_masks, masks)
            all_metrics.append(metrics)
    avg_metrics = {}
    for key in all_metrics[0].keys():
        avg_metrics[key] = np.mean([m[key] for m in all_metrics])
        avg_metrics[key + '_std'] = np.std([m[key] for m in all_metrics])
    return avg_metrics

def print_evaluation_results(metrics):
    print("\n📊 모델 성능 평가 결과")
    print("="*60)
    print(f"🎯 전체 성능:")
    print(f"  Pixel Accuracy: {metrics['pixel_accuracy']:.4f} ± {metrics['pixel_accuracy_std']:.4f}")
    print(f"  Mean IoU: {metrics['mean_iou']:.4f} ± {metrics['mean_iou_std']:.4f}")
    print(f"  Mean Dice: {metrics['mean_dice']:.4f} ± {metrics['mean_dice_std']:.4f}")
    print(f"\n📋 클래스별 성능:")
    print(f"  배경 (Background):")
    print(f"    IoU: {metrics['background_iou']:.4f} ± {metrics['background_iou_std']:.4f}")
    print(f"    Dice: {metrics['background_dice']:.4f} ± {metrics['background_dice_std']:.4f}")
    print(f"  로봇팔 (Robot Arm):")
    print(f"    IoU: {metrics['robot_arm_iou']:.4f} ± {metrics['robot_arm_iou_std']:.4f}")
    print(f"    Dice: {metrics['robot_arm_dice']:.4f} ± {metrics['robot_arm_dice_std']:.4f}")
    robot_iou = metrics['robot_arm_iou']
    if robot_iou >= 0.95:
        grade = "🥇 Excellent"
    elif robot_iou >= 0.90:
        grade = "🥈 Very Good"
    elif robot_iou >= 0.85:
        grade = "🥉 Good"
    elif robot_iou >= 0.80:
        grade = "✅ Acceptable"
    else:
        grade = "⚠️ Needs Improvement"
    print(f"\n🎖️ 로봇팔 성능 등급: {grade}")

def plot_key_metrics(metrics):
    """핵심 성능 지표 시각화"""
    keys = ['mean_iou', 'mean_dice', 'pixel_accuracy']
    labels = ['Mean IoU', 'Mean Dice', 'Pixel Accuracy']
    values = [metrics[k] for k in keys]
    stds = [metrics[k + '_std'] for k in keys]

    plt.figure(figsize=(8, 5))
    bars = plt.bar(labels, values, yerr=stds, capsize=5, color='skyblue')
    for bar, value in zip(bars, values):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, f"{value:.3f}",
                 ha='center', va='bottom', fontsize=11)
    plt.ylim(0, 1.05)
    plt.ylabel('Score')
    plt.title('SegFormer Evaluation Metrics')
    plt.grid(axis='y', linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.savefig('metric_bar_chart.png', dpi=150)
    plt.show()

def run_simple_evaluation():
    """간단한 평가 실행"""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    checkpoint_path = 'best_segformer_robot_arm.pth'

    # 데이터 로드
    original_images_root = '/home/ibom002/dataset'
    mask_images_root = '/home/ibom002/dataset'

    image_paths, mask_paths = match_image_mask_pairs(
        original_images_root, 
        mask_images_root
    )

    views = []
    for img_path in image_paths:
        if 'left' in img_path:
            views.append('left')
        elif 'right' in img_path:
            views.append('right')
        elif 'top' in img_path:
            views.append('top')
        else:
            views.append('unknown')

    from sklearn.model_selection import train_test_split
    train_img_paths, val_img_paths, train_mask_paths, val_mask_paths = train_test_split(
        image_paths, mask_paths,
        test_size=0.2,
        random_state=42,
        stratify=views
    )

    val_transforms = get_val_transforms(512)
    val_dataset = RobotArmDataset(val_img_paths, val_mask_paths, transform=val_transforms, is_train=False)
    from torch.utils.data import DataLoader
    val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=4, pin_memory=True)

    print(f"검증 데이터: {len(val_img_paths)}개")

    model, checkpoint = load_best_model(checkpoint_path, device)
    metrics = evaluate_model(model, val_loader, device)
    print_evaluation_results(metrics)
    plot_key_metrics(metrics)

    return metrics

# 실행
if __name__ == "__main__":
    metrics = run_simple_evaluation()
    print(f"\n🎊 평가 완료! 시각화 저장됨: metric_bar_chart.png")


In [ ]:
# ===============================
# 랜덤 샘플 모델 테스트 및 시각화
# ===============================

import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
from pathlib import Path
import albumentations as A
from albumentations.pytorch import ToTensorV2

def get_test_transforms(image_size=512):
    """테스트용 전처리 (정규화만)"""
    return A.Compose([
        A.Resize(image_size, image_size),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ])

def load_model_for_test(checkpoint_path, device):
    """테스트용 모델 로드"""
    print(f"📂 모델 로딩: {checkpoint_path}")
    
    model = SegFormerForRobotArm(num_classes=2, model_name="nvidia/mit-b2")
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    model = model.to(device)
    model.eval()
    
    print(f"✅ 모델 로드 완료 (Epoch {checkpoint['epoch']}, IoU: {checkpoint['val_iou']:.4f})")
    return model

def predict_single_image(model, image_path, mask_path, transform, device):
    """단일 이미지 예측"""
    # 원본 이미지 로드
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    original_image = image.copy()
    original_height, original_width = image.shape[:2]
    
    # 마스크 로드 (정답)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    mask = (mask > 127).astype(np.uint8)
    original_mask = mask.copy()
    
    # 전처리 (이미지와 마스크 모두 512x512로 리사이즈)
    transformed = transform(image=image, mask=mask)
    image_tensor = transformed['image'].unsqueeze(0).to(device)
    resized_mask = transformed['mask']  # 512x512로 리사이즈된 마스크
    
    # 예측
    with torch.no_grad():
        output = model(image_tensor)
        pred_mask = torch.argmax(output, dim=1)[0].cpu().numpy()
    
    # 예측 마스크를 원본 크기로 다시 리사이즈
    pred_mask_resized = cv2.resize(
        pred_mask.astype(np.uint8), 
        (original_width, original_height), 
        interpolation=cv2.INTER_NEAREST
    )
    
    # IoU 계산 (원본 크기에서)
    pred_robot = (pred_mask_resized == 1)
    true_robot = (original_mask == 1)
    intersection = np.logical_and(pred_robot, true_robot).sum()
    union = np.logical_or(pred_robot, true_robot).sum()
    iou = intersection / union if union > 0 else 1.0
    
    # Dice Score 계산 (원본 크기에서)
    dice = (2.0 * intersection) / (pred_robot.sum() + true_robot.sum()) if (pred_robot.sum() + true_robot.sum()) > 0 else 1.0
    
    return {
        'original_image': original_image,
        'true_mask': original_mask,
        'pred_mask': pred_mask_resized,  # 원본 크기로 복원된 예측 마스크
        'pred_mask_512': pred_mask,      # 512x512 예측 마스크 (시각화용)
        'iou': iou,
        'dice': dice,
        'image_path': image_path,
        'mask_path': mask_path,
        'original_size': (original_height, original_width)
    }

def random_sample_test(model, image_paths, mask_paths, num_samples=10, device='cuda'):
    """랜덤 샘플링으로 모델 테스트"""
    print(f"🎲 랜덤 샘플링 테스트 시작 ({num_samples}개 샘플)")
    
    # 랜덤 인덱스 선택
    total_samples = len(image_paths)
    random_indices = random.sample(range(total_samples), min(num_samples, total_samples))
    
    transform = get_test_transforms()
    results = []
    
    for i, idx in enumerate(random_indices):
        print(f"  처리 중: {i+1}/{len(random_indices)} - {Path(image_paths[idx]).name}")
        
        result = predict_single_image(
            model, 
            image_paths[idx], 
            mask_paths[idx], 
            transform, 
            device
        )
        results.append(result)
    
    return results

def visualize_test_results(results, save_path='test_results.png'):
    """테스트 결과 시각화"""
    num_samples = len(results)
    fig, axes = plt.subplots(num_samples, 3, figsize=(15, 5*num_samples))
    
    if num_samples == 1:
        axes = axes.reshape(1, -1)
    
    for i, result in enumerate(results):
        original_image = result['original_image']
        true_mask = result['true_mask']
        pred_mask = result['pred_mask']  # 원본 크기로 복원된 마스크
        iou = result['iou']
        dice = result['dice']
        original_size = result['original_size']
        
        # 원본 이미지
        axes[i, 0].imshow(original_image)
        axes[i, 0].set_title(f'Original Image {i+1}\n{Path(result["image_path"]).name}\nSize: {original_size[1]}x{original_size[0]}', fontsize=9)
        axes[i, 0].axis('off')
        
        # 실제 마스크
        axes[i, 1].imshow(true_mask, cmap='gray')
        axes[i, 1].set_title('Ground Truth Mask', fontsize=10)
        axes[i, 1].axis('off')
        
        # 예측 마스크 (원본 크기)
        axes[i, 2].imshow(pred_mask, cmap='gray')
        axes[i, 2].set_title(f'Predicted Mask\nIoU: {iou:.3f}, Dice: {dice:.3f}', fontsize=10)
        axes[i, 2].axis('off')
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"💾 결과 저장: {save_path}")

def visualize_test_results_comparison(results, save_path='test_comparison.png'):
    """원본 크기 vs 512x512 비교 시각화"""
    num_samples = min(len(results), 3)  # 최대 3개만 비교
    fig, axes = plt.subplots(num_samples, 4, figsize=(20, 5*num_samples))
    
    if num_samples == 1:
        axes = axes.reshape(1, -1)
    
    for i in range(num_samples):
        result = results[i]
        original_image = result['original_image']
        true_mask = result['true_mask']
        pred_mask_original = result['pred_mask']  # 원본 크기
        pred_mask_512 = result['pred_mask_512']   # 512x512
        iou = result['iou']
        
        # 원본 이미지
        axes[i, 0].imshow(original_image)
        axes[i, 0].set_title(f'Original Image {i+1}', fontsize=10)
        axes[i, 0].axis('off')
        
        # 실제 마스크
        axes[i, 1].imshow(true_mask, cmap='gray')
        axes[i, 1].set_title('Ground Truth', fontsize=10)
        axes[i, 1].axis('off')
        
        # 예측 마스크 (512x512)
        axes[i, 2].imshow(pred_mask_512, cmap='gray')
        axes[i, 2].set_title('Prediction (512x512)', fontsize=10)
        axes[i, 2].axis('off')
        
        # 예측 마스크 (원본 크기)
        axes[i, 3].imshow(pred_mask_original, cmap='gray')
        axes[i, 3].set_title(f'Prediction (Original)\nIoU: {iou:.3f}', fontsize=10)
        axes[i, 3].axis('off')
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"💾 비교 결과 저장: {save_path}")

def print_test_summary(results):
    """테스트 결과 요약 출력"""
    ious = [r['iou'] for r in results]
    dices = [r['dice'] for r in results]
    
    print(f"\n📊 테스트 결과 요약 ({len(results)}개 샘플)")
    print("="*60)
    print(f"IoU Score:")
    print(f"  평균: {np.mean(ious):.4f}")
    print(f"  표준편차: {np.std(ious):.4f}")
    print(f"  최소: {np.min(ious):.4f}")
    print(f"  최대: {np.max(ious):.4f}")
    
    print(f"\nDice Score:")
    print(f"  평균: {np.mean(dices):.4f}")
    print(f"  표준편차: {np.std(dices):.4f}")
    print(f"  최소: {np.min(dices):.4f}")
    print(f"  최대: {np.max(dices):.4f}")
    
    # 성능별 분류
    excellent = sum(1 for iou in ious if iou >= 0.95)
    good = sum(1 for iou in ious if 0.9 <= iou < 0.95)
    acceptable = sum(1 for iou in ious if 0.8 <= iou < 0.9)
    poor = sum(1 for iou in ious if iou < 0.8)
    
    print(f"\n🎯 성능 분포:")
    print(f"  Excellent (IoU ≥ 0.95): {excellent}개 ({excellent/len(results)*100:.1f}%)")
    print(f"  Good (0.9 ≤ IoU < 0.95): {good}개 ({good/len(results)*100:.1f}%)")
    print(f"  Acceptable (0.8 ≤ IoU < 0.9): {acceptable}개 ({acceptable/len(results)*100:.1f}%)")
    print(f"  Poor (IoU < 0.8): {poor}개 ({poor/len(results)*100:.1f}%)")

def analyze_by_view(results):
    """시점별 성능 분석"""
    view_results = {'left': [], 'right': [], 'top': [], 'unknown': []}
    
    for result in results:
        path = result['image_path']
        if 'left' in path:
            view = 'left'
        elif 'right' in path:
            view = 'right'
        elif 'top' in path:
            view = 'top'
        else:
            view = 'unknown'
        
        view_results[view].append(result['iou'])
    
    print(f"\n📊 시점별 성능 분석:")
    print("-"*40)
    for view, ious in view_results.items():
        if ious:
            print(f"{view.upper()}: {len(ious)}개 - 평균 IoU {np.mean(ious):.4f}")

def run_random_test(num_samples=10):
    """랜덤 테스트 메인 함수"""
    print("🚀 랜덤 샘플 모델 테스트 시작")
    print("="*60)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # 1. 데이터 경로 수집
    print("📊 데이터 수집 중...")
    original_images_root = '/home/ibom002/dataset'
    mask_images_root = '/home/ibom002/dataset'
    
    image_paths, mask_paths = match_image_mask_pairs(
        original_images_root, 
        mask_images_root
    )
    
    print(f"총 {len(image_paths)}개 이미지-마스크 쌍 발견")
    
    # 2. 모델 로드
    model = load_model_for_test('best_segformer_robot_arm.pth', device)
    
    # 3. 랜덤 샘플링 테스트
    results = random_sample_test(model, image_paths, mask_paths, num_samples, device)
    
    # 4. 결과 시각화
    print(f"\n🖼️ 결과 시각화 중...")
    visualize_test_results(results, f'random_test_{num_samples}_samples.png')
    
    # 4-1. 추가: 크기 비교 시각화 (처음 3개 샘플)
    if len(results) >= 3:
        print(f"🔍 크기 비교 시각화 중...")
        visualize_test_results_comparison(results[:3], f'size_comparison_{num_samples}_samples.png')
    
    # 5. 결과 분석
    print_test_summary(results)
    analyze_by_view(results)
    
    print(f"\n🎊 테스트 완료!")
    
    return results

# ===============================
# 특정 시점만 테스트하는 함수
# ===============================

def test_specific_view(view='left', num_samples=5):
    """특정 시점만 테스트"""
    print(f"🎯 {view.upper()} 시점 전용 테스트")
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # 데이터 수집
    original_images_root = '/home/ibom002/dataset'
    mask_images_root = '/home/ibom002/dataset'
    image_paths, mask_paths = match_image_mask_pairs(original_images_root, mask_images_root)
    
    # 특정 시점만 필터링
    view_image_paths = []
    view_mask_paths = []
    
    for img_path, mask_path in zip(image_paths, mask_paths):
        if view in img_path:
            view_image_paths.append(img_path)
            view_mask_paths.append(mask_path)
    
    print(f"{view.upper()} 시점 데이터: {len(view_image_paths)}개")
    
    # 모델 로드 및 테스트
    model = load_model_for_test('best_segformer_robot_arm.pth', device)
    results = random_sample_test(model, view_image_paths, view_mask_paths, num_samples, device)
    
    # 시각화
    visualize_test_results(results, f'{view}_view_test.png')
    print_test_summary(results)
    
    return results

# ===============================
# 실행 부분
# ===============================

if __name__ == "__main__":
    # 랜덤 10개 샘플 테스트
    results = run_random_test(num_samples=10)
    
    # 또는 특정 시점만 테스트
    # left_results = test_specific_view('left', num_samples=5)
    # right_results = test_specific_view('right', num_samples=5)
    # top_results = test_specific_view('top', num_samples=5)

원본이미지에 마스크 채우기

In [ ]:
import time
import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms
from transformers import SegformerForSemanticSegmentation
from pathlib import Path
import torch.nn as nn
import albumentations as A
from albumentations.pytorch import ToTensorV2

# ===============================
# SegFormer 모델 정의
# ===============================

class SegFormerForRobotArm(nn.Module):
    def __init__(self, num_classes=2, model_name="nvidia/mit-b2"):
        super().__init__()
        self.num_classes = num_classes
        
        # SegFormer 모델 로드 (pretrained MiT 백본 포함)
        # model_name 옵션:
        # - nvidia/mit-b0: 3.7M params (빠름, 가벼움)
        # - nvidia/mit-b1: 14M params  
        # - nvidia/mit-b2: 25M params (권장, 균형점)
        # - nvidia/mit-b3: 45M params (더 높은 성능)
        # - nvidia/mit-b4: 62M params
        # - nvidia/mit-b5: 82M params (최고 성능, 느림)
        
        print(f"🏗️ SegFormer 모델 로딩: {model_name}")
        self.segformer = SegformerForSemanticSegmentation.from_pretrained(
            model_name,
            num_labels=num_classes,
            ignore_mismatched_sizes=True
        )
        
        # 모델 파라미터 수 출력
        total_params = sum(p.numel() for p in self.segformer.parameters())
        trainable_params = sum(p.numel() for p in self.segformer.parameters() if p.requires_grad)
        print(f"📊 총 파라미터: {total_params:,}")
        print(f"📊 훈련 가능 파라미터: {trainable_params:,}")
        
    def forward(self, pixel_values):
        outputs = self.segformer(pixel_values=pixel_values)
        logits = outputs.logits
        
        # SegFormer 출력을 입력 크기로 업샘플링
        # 입력: [B, C, H, W] -> 출력: [B, num_classes, H, W]
        upsampled_logits = nn.functional.interpolate(
            logits,
            size=pixel_values.shape[-2:],  # (H, W)
            mode='bilinear',
            align_corners=False
        )
        
        return upsampled_logits

model = SegFormerForRobotArm(num_classes=2, model_name="nvidia/mit-b2")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def overlay_mask_on_image(image: np.ndarray, mask: np.ndarray, alpha=0.5, color=(0, 255, 0)):
    """원본 이미지 위에 초록색 마스크를 덧씌움"""
    color_mask = np.zeros_like(image)
    color_mask[mask == 1] = color
    blended = cv2.addWeighted(image, 1 - alpha, color_mask, alpha, 0)
    return blended

def get_val_transforms(image_size=512):
    return A.Compose([
        A.Resize(image_size, image_size),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ])
    
def visualize_from_saved_model(image_path, model_path='best_segformer_robot_arm.pth', model_size='nvidia/mit-b2', image_size=512):
    # 1. 이미지 불러오기
    image = Image.open(image_path).convert("RGB")
    orig_np = np.array(image)

    # 2. 변환 적용
    transform = get_val_transforms(image_size)
    transformed = transform(image=np.array(image))
    input_tensor = transformed['image'].unsqueeze(0).to(device)

    # 3. 모델 로드
    model = SegFormerForRobotArm(num_classes=2, model_name=model_size)
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    model = model.to(device)
    model.eval()

    # 4. 추론 시간 측정
    with torch.no_grad():
        start = time.time()
        output = model(input_tensor)  # [1, C, H, W]
        end = time.time()

        pred_mask = torch.argmax(output, dim=1).squeeze(0).cpu().numpy().astype(np.uint8)

    print(f"⏱️ 추론 시간: {(end - start)*1000:.2f} ms")  # 또는 sec 단위로 보기 원하면 : .4f sec

    # 5. 시각화
    if orig_np.shape[:2] != pred_mask.shape:
        pred_mask = cv2.resize(pred_mask, (orig_np.shape[1], orig_np.shape[0]), interpolation=cv2.INTER_NEAREST)

    overlay = overlay_mask_on_image(orig_np, pred_mask, alpha=0.5, color=(0, 255, 0))

    plt.figure(figsize=(10, 5))
    plt.imshow(overlay)
    plt.title("SegFormer result")
    plt.axis('off')
    plt.show()

# 예시 사용
image_path = "/home/ibom002/dataset/Fr5_intertek_4th_250526/left/zed_38007749_left_1748249115.548.jpg"
visualize_from_saved_model(image_path)
